In [27]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
import glob
import re
#import xmltodict
import pickle
import untangle
import uuid
from tqdm import tqdm
from decimal import Decimal
from skimage import measure, morphology
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.segmentation import clear_border
from skimage.filters import roberts, sobel

from mpl_toolkits.mplot3d.art3d import Poly3DCollection

DATA_PATH = '/kaggle_2/lidc_idri/data/'
DATA_PATH_XML = '/kaggle_2/lidc_idri/data/tcia-lidc-xml/'
DATA_PATH_SCANS = '/kaggle_2/lidc_idri/data/LIDC/DOI/'
DATA_PATH_POST_PROCESSED_SCANS = '/kaggle_2/lidc_idri/data/scans_resampled_unsegmented/'
DATA_PATH_NODULES = '/kaggle_2/lidc_idri/data/nodules_chunked/'
CHUNK_SIZE = 32

In [29]:
with open(DATA_PATH + 'patient_scans_map.pkl', 'rb') as f:
    patient_scans_map = pickle.load(f)

with open(DATA_PATH + 'patient_nodules_map.pkl', 'rb') as f:
    patient_nodules_map = pickle.load(f)
    
with open(DATA_PATH + 'patient_deduped_nodules_map.pkl', 'rb') as f:
    patient_deduped_nodules_map = pickle.load(f)

In [30]:
print(len(patient_scans_map.keys()))
print(len(patient_nodules_map.keys()))
print(len(patient_deduped_nodules_map.keys()))

1018
998
998


In [31]:
# Load the scans in given folder path
def load_scan(paths):
    slices = [dicom.read_file(path) for path in paths]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
    
    origin = np.array(list(reversed(slices[0].ImagePositionPatient)), dtype=np.float32)
    
    # Determine current pixel spacing
    spacing = np.array([slices[0].SliceThickness] + slices[0].PixelSpacing, dtype=np.float32)

    return slices, origin, spacing

def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

In [32]:
weird_chunks = {}
weird_chunk_count = 0
RESIZE_SPACING = [1,1,1]
items = list(patient_deduped_nodules_map.items())
for idx in tqdm(range(len(items))):
    patient_id = items[idx][0]
    nodules = items[idx][1]
#    print(patient_id)
    patient_scan_files = patient_scans_map[patient_id]['scans']
    patient_scan_files.sort()
    scan, origin, spacing = load_scan(patient_scan_files)
    scan_resampled = np.load(DATA_PATH_POST_PROCESSED_SCANS + "scan_%s.npy" % (patient_id))
#    print('Original scan', (len(scan), 512, 512))
#    print('Resampled scan', scan_resampled.shape)
#    print('Nodules',len(nodules))
    #print(nodules)
    #print('---')
    #print(patient_nodules_map[patient_id])

    X = np.ndarray((len(nodules), CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), dtype=np.float32)
    Y = np.ndarray([len(nodules), 1], dtype=np.float32)
    count = 0
    for nodule in nodules:
        coords = nodule['coords']
        coords_np = np.ndarray((len(coords), 4), dtype=np.float32)
        for idx in range(len(coords)):
            coords_np[idx] = coords[idx]
        #print(coords_np)
        minZ = np.amin(coords_np[:, 0], axis=0)
        maxZ = np.amax(coords_np[:, 0], axis=0)
        minY = np.amin(coords_np[:, 1], axis=0)
        maxY = np.amax(coords_np[:, 1], axis=0)
        minX = np.amin(coords_np[:, 2], axis=0)
        maxX = np.amax(coords_np[:, 2], axis=0)
        
        centerZ = int((minZ + maxZ) / 2.0)
        centerY = int((minY + maxY) / 2.0)
        centerX = int((minX + maxX) / 2.0)

        imageCoord = np.array((centerZ, centerY, centerX))
        
#        print('Pre process nodule coords', imageCoord)
        imageCoord = world_2_voxel(imageCoord, origin, RESIZE_SPACING)
#        print('Post process nodule coords', imageCoord)        
        
        Z1 = int(imageCoord[2]) - int(CHUNK_SIZE/2)
        Z2 = int(imageCoord[2]) + int(CHUNK_SIZE/2)
        Y1 = int(imageCoord[1]) - int(CHUNK_SIZE/2)
        Y2 = int(imageCoord[1]) + int(CHUNK_SIZE/2)
        X1 = int(imageCoord[0]) - int(CHUNK_SIZE/2)
        X2 = int(imageCoord[0]) + int(CHUNK_SIZE/2)
        
        X1 = 0 if (X1 < 0) else X1
        Y1 = 0 if (Y1 < 0) else Y1
        Z1 = 0 if (Z1 < 0) else Z1
        
        X2 = scan_resampled.shape[2] if (X2 > scan_resampled.shape[2]) else X2
        Y2 = scan_resampled.shape[1] if (Y2 > scan_resampled.shape[1]) else Y2
        Z2 = scan_resampled.shape[0] if (Z2 > scan_resampled.shape[0]) else Z2
        
#       print(int(minZ), int(maxZ), int(minY), int(maxY), int(minX), int(maxX))
#       print(int(centerZ), int(centerY), int(centerX))
#        print(Z1, Z2, Y1, Y2, X1, X2)
        
        if (Z2 > scan_resampled.shape[0] or Y2 > scan_resampled.shape[1] or X2 > scan_resampled.shape[2] or Z2 < Z1 or Y2 < Y1 or X2 < X1):
            print('Found weird chunk!')
            if patient_id in weird_chunks.keys():
                weird_chunks[patient_id].append({'shape': scan_resampled.shape, 'center': [centerZ, centerY, centerX], 'chunk_coords': [Z1, Z2, Y1, Y2, X1, X2]})
            else:
                weird_chunks[patient_id] = []
                weird_chunks[patient_id].append({'shape': scan_resampled.shape, 'center': [centerZ, centerY, centerX], 'chunk_coords': [Z1, Z2, Y1, Y2, X1, X2]})
            weird_chunk_count += 1
            continue
        
        chunk = np.full((CHUNK_SIZE, CHUNK_SIZE, CHUNK_SIZE), -1000.0, np.float32)
        chunk[0:Z2-Z1, 0:Y2-Y1, 0:X2-X1] = scan_resampled[Z1:Z2,Y1:Y2,X1:X2]

        X[count,:,:,:] = chunk
        if 'malignancy' in nodule:
            Y[count,] = nodule['malignancy']
        else:
            Y[count,] = 0.0
        count = count + 1
    np.save(DATA_PATH_NODULES + patient_id + '_X.npy', X)
    np.save(DATA_PATH_NODULES + patient_id + '_Y.npy', Y)

  1%|          | 7/998 [02:53<7:33:03, 27.43s/it] 

Found weird chunk!
Found weird chunk!


  1%|▏         | 14/998 [06:02<7:10:58, 26.28s/it]

Found weird chunk!
Found weird chunk!


  2%|▏         | 21/998 [08:14<5:33:58, 20.51s/it]

...

 97%|█████████▋| 965/998 [6:14:13<12:26, 22.62s/it]

Found weird chunk!


 99%|█████████▉| 992/998 [6:22:56<01:51, 18.61s/it]

Found weird chunk!
Found weird chunk!
Found weird chunk!


100%|██████████| 998/998 [6:26:25<00:00, 30.62s/it]


In [33]:
# Do NOT run this cell as it will overwrite already existing weird_chunks file on disk
# with open(DATA_PATH + 'weird_chunks.pkl', 'wb') as f:
#     pickle.dump(weird_chunks, f, pickle.HIGHEST_PROTOCOL)
    
# print('Saved weird_chunks map!')

Saved weird_chunks map!


In [34]:
with open(DATA_PATH + 'weird_chunks.pkl', 'rb') as f:
    weird_chunks = pickle.load(f)

In [37]:
total_weird_chunks = 0
for p_id, chunks in weird_chunks.items():
    total_weird_chunks += len(chunks)

total_weird_chunks = 0
for p_id, nodules in patient_deduped_nodules_map.items():
    total_weird_chunks += len(nodules)
total_weird_chunks

8419